# This notebook attempts to extract all possible accounts from three *annual* financial reports: the balance sheet, income statement and cash flow statement for cafeF.

### Why this work?
The output of this work will be used for a table mapping a cafeF account to a hard-coded FAD account, which will be stored in our database.

### How is a cafeF account defined?
We define a cafeF account as ...

In [22]:
import pandas as pd
import json
import glob
import re
import traceback

### Here we define a function to simplify account names

In [23]:
from functions_cafef.fad_crawl_cafef.helpers.processingData import simplifyText

In [24]:
def removeDigit(s):
    """Remove all digits from string
    """
    pattern = '[0-9]'
    s = re.sub(pattern, '', s)
    return s

def findParentName(id_, l_):
    id_split = id_[:-1].split("0")[0]
    for e in l_:
        if id_split == e[0]:
            return removeDigit(simplifyText(e[1]))
    return "N/A"

### Define all necessary dictionaries

In [25]:
report_types = {"BS":{},
                "IS":{},
                "CF":{}}
lookup_dict_all = {"BS":{},
                    "IS":{},
                    "CF": {}}

### Process each report type sequentially
Before beginning the code, it's helpful to see what a BS report file looks like:

```
{
    "data": [
        [
            "100",
            "I - TÀI SẢN NGẮN HẠN",
            "186,834,158,952",
            "225,328,146,085",
            "260,649,376,048",
            "218,590,766,332"
        ],
        [
            "110",
            "1. Tiền và các khoản tương đương tiền",
            "9,348,259,155",
            "53,209,096,419",
            "54,853,929,411",
            "13,608,853,476"
        ],
        [
            "111",
            "1.1.Tiền",
            "9,348,259,155",
            "53,209,096,419",
            "54,853,929,411",
            "13,608,853,476"
        ],
        [
            "112",
            "1.2.Các khoản tương đương tiền",
            "",
            "",
            "",
            ""
        ]],
    "periods": [
        "quý 1 2019",
        "quý 2 2019",
        "quý 3 2019",
        "quý 4 2019"
    ]
}
```

### Let's see how many unique balance sheet accounts there are

In [46]:
### Let's try with BS first
### Accounts in cafeF should be got rid of numbers
report_type = "BS"
result = []
for file in glob.glob(f'functions_cafef/schemaData/BS_cafef/*{report_type}*.json'):
    with open(file, 'r') as jsonfile:
        try:
            j = json.load(jsonfile)
            for e in j['data']:
                e[0] = e[0].strip("0")
                e[1] = simplifyText(removeDigit(e[1]))
#             print(j['data'])
            for i, acc in enumerate(j['data']):
#                 prev_acc = j['data'][max(0, i-1)][1]
#                 next_acc = j['data'][min(len(j['data'])-1, i+1)][1]
                
                acc_id = acc[0]
                current_acc = acc[1]
                parent_acc = findParentName(acc_id, j['data'])
                entry = f'{current_acc};{parent_acc}'
                
                if entry not in result:
                    result.append(entry)
        except Exception as e:
            print(traceback.format_exc())

print(len(result))
for a in result:
    print(a)

149
;i tài sản ngắn hạn;N/A
;tiền và các khoản tương đương tiền;i tài sản ngắn hạn
;tiền;tiền và các khoản tương đương tiền
;các khoản tương đương tiền;tiền và các khoản tương đương tiền
;các khoản đầu tư tài chính ngắn hạn;i tài sản ngắn hạn
;chứng khoán kinh doanh;các khoản đầu tư tài chính ngắn hạn
;dự phòng giảm giá chứng khoán kinh doanh;các khoản đầu tư tài chính ngắn hạn
;đầu tư nắm giữ đến ngày đáo hạn;các khoản đầu tư tài chính ngắn hạn
;các khoản phải thu ngắn hạn;i tài sản ngắn hạn
;phải thu khách hàng;các khoản phải thu ngắn hạn
;trả trước cho người bán;các khoản phải thu ngắn hạn
;phải thu nội bộ ngắn hạn;các khoản phải thu ngắn hạn
;phải thu theo tiến độ kế hoạch hợp đồng xây dựng;các khoản phải thu ngắn hạn
;phải thu về cho vay ngắn hạn;các khoản phải thu ngắn hạn
;các khoản phải thu khác;các khoản phải thu ngắn hạn
;dự phòng phải thu ngắn hạn khó đòi;các khoản phải thu ngắn hạn
;tài sản thiếu chờ xử lý;dự phòng phải thu ngắn hạn khó đòi
;hàng tồn kho;i tài sản ngắn hạn


### See how many duplicate accounts based on `acc name` and `parent acc name`

In [48]:
n_parent = []
for a in result:
    n = ";".join(a.split(";")[1:])
    if n in n_parent:
        print(a)
    else:
        n_parent.append(n)

### Let's see how many tickers we got

In [28]:
tickers = set()
for file in glob.glob(f'functions_cafef/schemaData/BS_cafef/*{report_type}*.json'):
    ticker = file.split("/")[-1].split("_")[0]
    tickers.add(ticker)
print(len(tickers))
print((tickers))

1850
{'HPU', 'UEM', 'QNS', 'RESCO', 'NASICO', 'FID', 'DLV', 'CCH', 'VINASUGAR1', 'BHT', 'IFS', 'QCG', 'DLD', 'UDJ', 'TTZ', 'KHA', 'THMILK', 'MKP', 'CPC', 'PFL', 'ASIAFOOD', 'HPW', 'SDA', 'SPH', 'VTC', 'VDP', 'DVC', 'EMS', 'LBE', 'SD3', 'PRO', 'SAIGONPETRO', 'PPI', 'PXM', 'USC', 'TOP', 'EVG', 'TIC', 'STK', 'RTB', 'C47', 'MIPEC', 'VC1', 'DXG', 'DC2', 'DFC', 'CSG', 'PLC', 'CDR', 'DBD', 'HNI', 'CRC', 'LAW', 'VICEM', 'SPCT', 'NS2', 'DCL', 'HDC', 'SSU', 'AQN', 'TNP', 'YBM', 'ABT', 'NTH', 'TCO', 'LGC', 'SBA', 'DCT', 'DNC', 'SGP', 'VINAMOTOR', 'SB1', 'SHA', 'VISSAI', 'AGF', 'GLW', 'PTB', 'GTA', 'MCO', 'SVI', 'HVG', 'C12', 'SDB', 'PCE', 'VTP', 'SDY', 'RAT', 'CEO', 'GAMI', 'PEN', 'SAIGONCONS', 'PTT', 'DOP', 'FSO', 'PNJ', 'LM8', 'SSN', 'TMHG', 'LTG', 'SLS', 'MML', 'DNW', 'SGS', 'PND', 'KTT', 'TQW', 'DAP', 'VNRAILWAYS', 'VHW', 'VLA', 'LIC', 'PHN', 'SGH', 'TAG', 'VKC', 'DTC', 'CQN', 'HES', 'VMA', 'TC6', 'ASM', 'NPH', 'DVIZ', 'POM', 'VPL', 'GCB', 'TA6', 'HAP', 'QLD', 'HHC', 'IJC', 'ILA', 'TDS', 'SWC

In [29]:
# report_types_expressdetails = {"CDKT":{},
#                                 "KQKD":{},
#                                 "LC":{}}
# lookup_dict_all_expressdetails = {"CDKT":{},
#                                     "KQKD":{},
#                                     "LC": {}}

# with open('functions/schema/bizType_ind_list.json', 'r') as jsonfile:
#     biztypes_inds = sorted(json.load(jsonfile))
#     jsonfile.close()

# for report_type, report_all_accounts in report_types_expressdetails.items():      
#     for biztype_ind in biztypes_inds:
#         biztype, ind = biztype_ind.split(";")[0], biztype_ind.split(";")[1]
#         if biztype_ind not in lookup_dict_all_expressdetails[report_type].keys():
#             lookup_dict_all_expressdetails[report_type][biztype_ind] = {}
#         if biztype_ind not in report_types_expressdetails[report_type].keys():
#             report_types_expressdetails[report_type][biztype_ind] = {}
#         for file in glob.glob(f'functions/schemaData/financeInfo/{biztype}_{ind}*{report_type}_Annual*.json'):
#             with open(file, 'r') as jsonfile:
#                 try:
#                     j = json.load(jsonfile)
#                     for report_fullname, report_content in j[1].items():
#                         if report_fullname not in lookup_dict_all_expressdetails[report_type][biztype_ind].keys():
#                             lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname] = {}
#                         for content in report_content:
#                             acc_id = content['ReportNormID']
#                             if acc_id not in lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname].keys():
#                                 lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname][acc_id] = content
                            
#                             ### Test if any of the three elements of the entry are ""
# #                             acc_n = simplify_text(content['NameEn'])
# #                             acc_vi_n = simplify_text(content['Name'])
# #                             acc_parent_id = content['ParentReportNormID']
# #                             if (acc_n == "" and acc_vi_n == "") or (acc_n == "" and acc_parent_id == "") or (acc_vi_n == "" and acc_parent_id == ""):
# #                                 print(file)
# #                                 print(f'{acc_n};{acc_vi_n};{acc_parent_id}')
                                
#                 except Exception as e:
#                     print(f'EXCEPTION: {e}')
                    
#         for report_fullname, report_content in lookup_dict_all_expressdetails[report_type][biztype_ind].items():
#             if report_fullname not in report_all_accounts[biztype_ind].keys():
#                 report_all_accounts[biztype_ind][report_fullname] = {}
#             for acc_id, content in report_content.items():
#                 acc_n = simplify_text(content['NameEn'])
#                 acc_vi_n = simplify_text(content['Name'])
#                 parent_n = simplify_text(lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname][content['ParentReportNormID']]['NameEn'])
#                 parent_vi_n = simplify_text(lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname][content['ParentReportNormID']]['Name'])
                
#                 entry = f'{acc_n};{parent_n};{acc_vi_n};{parent_vi_n}'
#                 if entry not in report_all_accounts[biztype_ind][report_fullname].keys():
#                     report_all_accounts[biztype_ind][report_fullname][entry] = [acc_id]
#                 else:
#                     report_all_accounts[biztype_ind][report_fullname][entry].append(acc_id)
                    
#             for entry, ids in report_all_accounts[biztype_ind][report_fullname].items():
#                 if len(ids) > 1:
#                     print("=== WARNING: one entry has multiple account IDs ===")
#                     print(f'ENTRY "{entry}", BIZ TYPE;INDUSTRY "{biztype_ind}", REPORT "{report_type}-{report_fullname}" \n')

#             with open(f'functions/schema/{biztype_ind}_{report_type}_{report_fullname}_all_accounts.json', 'w') as writefile:
#                 json.dump(report_all_accounts[biztype_ind], writefile, ensure_ascii=False, indent=4)